

# Agents

An agent is a software entity that communicates via messages, maintains its own state, and performs actions in response to received messages or changes in its state. 

AgentChat provides a set of preset Agents, each with variations in how an agent might respond to received messages. 

Each agent inherits from a `BaseChatAgent` class with a few generic properties:

- `name`: The name of the agent. This is used by the team to uniquely identify the agent. It should be unique within the team.
- `description`: The description of the agent. This is used by the team to make decisions about which agents to use. The description should detail the agent's capabilities and how to interact with it.
 
```{tip}
How do agents send and receive messages? 

AgentChat is built on the `autogen-core` package, which handles the details of sending and receiving messages. `autogen-core` provides a runtime environment, which facilitates communication between agents (message sending and delivery), manages their identities and lifecycles, and enforces security and privacy boundaries. 
AgentChat handles the details of instantiating a runtime and registering agents with the runtime.

To learn more about the runtime in `autogen-core`, see the [autogen-core documentation on agents and runtime](../../core-user-guide/framework/agent-and-agent-runtime.ipynb).
```

Each agent also implements an `on_message` method that defines the behavior of the agent in response to a message.

## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond.

In [1]:

import logging
from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.agents import  CodingAssistantAgent, ToolUseAssistantAgent, TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, MaxMessageTermination
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code." ,
    model_client=model_client,
)

In [2]:
result = await writing_assistant_agent.on_messages(messages=[
    TextMessage(content="What is the weather right now in France?", source="user"),
], cancellation_token=None)
print(result)

source='writing_assistant_agent' content="I'm unable to provide real-time data, including current weather conditions. To find out the current weather in France, you can check a reliable weather website or use a weather app on your phone. Websites like the BBC Weather, Weather.com, or the French meteorological service Météo-France would have up-to-date information."




## ToolUseAssistantAgent

This agent responds to messages by making appropriate tool or function calls.

```{tip}
Understanding Tool Calling

Large Language Models (LLMs) are typically limited to generating text or code responses. However, many complex tasks benefit from the ability to use external tools that perform specific actions, such as fetching data from APIs or databases.

To address this limitation, modern LLMs can now accept a list of available tool schemas (descriptions of tools and their arguments) and generate a tool call message. This capability is known as **Tool Calling** or **Function Calling** and is becoming a popular pattern in building intelligent agent-based applications.

For more information on tool calling, refer to the documentation from [OpenAI](https://platform.openai.com/docs/guides/function-calling) and [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use).
```

To set up a ToolUseAssistantAgent in AgentChat, follow these steps:

1. Define the tool, typically as a Python function.
2. Wrap the function in the `FunctionTool` class from the `autogen-core` package. This ensures the function schema can be correctly parsed and used for tool calling.
3. Attach the tool to the agent.
 

In [3]:
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

tool_use_agent = ToolUseAssistantAgent(
    "tool_use_agent",
    system_message="You are a helpful assistant that solves tasks by only using your tools.",
    model_client=model_client,
    registered_tools=[get_weather_tool],
)

In [4]:
tool_result = await tool_use_agent.on_messages(messages=[
    TextMessage(content="What is the weather right now in France?", source="user"),
], cancellation_token=None)
print(tool_result)

source='tool_use_agent' content="France is a large country with various regions experiencing different weather conditions at any given time. Could you please specify a city in France for which you'd like to know the current weather?"


We can see that the response generated by the ToolUseAssistantAgent is a tool call message which can then be executed to get the right result. 



## CodeExecutionAgent 

This agent preset extracts and executes code snippets found in received messages and returns the output. It is typically used within a team where a `CodingAssistantAgent` is also present - the `CodingAssistantAgent` can generate code snippets, which the `CodeExecutionAgent` receives and executes to make progress on a task. 

```{note}
It is recommended that the `CodeExecutionAgent` uses a Docker container to execute code snippets. This ensures that the code snippets are executed in a safe and isolated environment. 

To use Docker, your environment must have Docker installed and running.
```

In the code snippet below, we show how to set up a `CodeExecutionAgent` that uses the `DockerCommandLineCodeExecutor` class to execute code snippets in a Docker container. The `work_dir` parameter indicates where all executed files are first saved locally before being executed in the Docker container.

In [5]:
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors import DockerCommandLineCodeExecutor

# type: ignore[syntax]
async with DockerCommandLineCodeExecutor(work_dir="coding") as code_executor:
    code_executor_agent = CodeExecutorAgent(
        "code_executor", code_executor=code_executor)
    code_execution_result = await code_executor_agent.on_messages(messages=[
        TextMessage(content="Here is some code \n ```python print('Hello world') \n``` ",
                    source="user"),
    ], cancellation_token=None)
    print(code_execution_result)

source='code_executor' content='No code blocks found in the thread.'


## Summary
So far, we have learned a few key concepts:

- How to define agents and the types of agents supported in AgentChat 
- How to send messages to agents by calling the `on_message` method and viewing the agent's response 

However, the ability to address complex tasks is often best served by groups of agents that interact as a team. Let us review how to build these teams.